In [68]:
import numpy as np
import pandas as pd

In [69]:
df=pd.read_excel('Test_params.xlsx')
df

,Area,Solar_cost,Wind_cost,Solar_Power,Wind_Power,lat,long
0,401,1.141326e+10,1.331306e+10,790.088322,569.477760,8.235490,77.550294
1,686,1.954698e+10,2.186692e+10,852.281140,1172.791598,27.231708,70.303710
2,735,2.208054e+10,2.544518e+10,749.719273,735.282807,30.825460,78.967529
3,487,1.375784e+10,1.627365e+10,784.336841,761.489126,13.070384,80.200195
4,869,2.274979e+10,2.711702e+10,800.376047,848.454601,15.621395,73.876465
5,679,1.857937e+10,2.218504e+10,857.837854,515.095283,21.824384,70.839844
6,966,2.734503e+10,3.168682e+10,798.702382,650.341881,17.668997,79.586060


In [70]:
lat=df['lat']
long=df['long']
lat

0     8.235490
1    27.231708
2    30.825460
3    13.070384
4    15.621395
5    21.824384
6    17.668997
Name: lat, dtype: float64

In [71]:
p_s=np.array(df['Solar_Power'])
p_w=np.array(df['Wind_Power'])
p=np.column_stack([p_s,p_w])
p

array([[ 790.08832198,  569.47776024],
       [ 852.28113985, 1172.79159789],
       [ 749.71927258,  735.28280668],
       [ 784.33684082,  761.48912612],
       [ 800.37604668,  848.45460067],
       [ 857.83785422,  515.09528341],
       [ 798.70238179,  650.3418809 ]])

In [72]:
c_s=np.array(df['Solar_cost'])
c_w=np.array(df['Wind_cost'])
c=np.column_stack([c_s,c_w])
c

array([[1.14132636e+10, 1.33130644e+10],
       [1.95469819e+10, 2.18669223e+10],
       [2.20805433e+10, 2.54451813e+10],
       [1.37578353e+10, 1.62736454e+10],
       [2.27497912e+10, 2.71170200e+10],
       [1.85793673e+10, 2.21850431e+10],
       [2.73450311e+10, 3.16868234e+10]])

In [73]:
a=np.array(df['Area'])
a

array([401, 686, 735, 487, 869, 679, 966], dtype=int64)

In [74]:
n=df.shape[0]
t=2
m=3   #Number of power plants you need to built
b=10e10 #Budget

def Calculate_power(model,i,j):
    return p[i-1][j-1]*a[i-1]
def Calculate_cost(model,i,j):
    return c[i-1][j-1]

## Create a model
model = ConcreteModel()

## Create parameters and sets
model.n = n
model.t = t
model.types = Set(initialize=range(1, t+1))
model.location = Set(initialize=range(1, n+1))
model.power = Param(model.location,model.types, initialize=Calculate_power)
model.cost= Param(model.location,model.types, initialize=Calculate_cost)

## Create the variables
model.x = Var(model.location, model.types, domain=Binary)

In [75]:
#Objective Function
model.totalpower = Objective(expr=sum(model.power[i, j]*model.x[i, j] 
                    for i in model.location for j in model.types),sense=maximize)

In [76]:
## Assignment Constraints
model.assignment = ConstraintList()
for i in model.location:
  model.assignment.add(expr=sum(model.x[i, j] for j in model.types)<=1)
model.cons1=Constraint(expr = sum(model.cost[i, j]*model.x[i, j] for i in model.location for j in model.types) <= b)
model.cons2=Constraint(expr = sum(model.x[i, j] for i in model.location for j in model.types)==m)

In [77]:
#Solving
solver = SolverFactory('cbc.exe')
solver.solve(model)

# display solution
print('\nObjective Function = ', model.totalpower())

model.x.display()
model.cons1()
model.cons2()



Objective Function =  2313388.584939813
x : Size=14, Index=x_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :     0 :   0.0 :     1 : False : False : Binary
    (1, 2) :     0 :   0.0 :     1 : False : False : Binary
    (2, 1) :     0 :   0.0 :     1 : False : False : Binary
    (2, 2) :     0 :   1.0 :     1 : False : False : Binary
    (3, 1) :     0 :   0.0 :     1 : False : False : Binary
    (3, 2) :     0 :   0.0 :     1 : False : False : Binary
    (4, 1) :     0 :   0.0 :     1 : False : False : Binary
    (4, 2) :     0 :   0.0 :     1 : False : False : Binary
    (5, 1) :     0 :   0.0 :     1 : False : False : Binary
    (5, 2) :     0 :   1.0 :     1 : False : False : Binary
    (6, 1) :     0 :   0.0 :     1 : False : False : Binary
    (6, 2) :     0 :   0.0 :     1 : False : False : Binary
    (7, 1) :     0 :   1.0 :     1 : False : False : Binary
    (7, 2) :     0 :   0.0 :     1 : False : False : Binary


3.0

In [79]:
decider=pd.DataFrame()

In [80]:
d=model.x.extract_values()
latitude=[]
longitude=[]
choice=[]
cost=[]
area=[]
expenses=0
for key in d:
  if(d[key]==1 and key[1]==1):
    latitude.append(lat[key[0]-1])
    longitude.append(long[key[0]-1])
    choice.append("Solar")
    cost.append(c_s[key[0]-1])
    area.append(a[key[0]-1])
    expenses+=c_s[key[0]-1]
  elif(d[key]==1 and key[1]==2):
    latitude.append(lat[key[0]-1])
    longitude.append(long[key[0]-1])
    choice.append("Wind")
    cost.append(c_w[key[0]-1])
    area.append(a[key[0]-1])
    expenses+=c_w[key[0]-1]
decider['lat']=latitude
decider['long']=longitude
decider['Choice']=choice
decider['Cost']=cost
decider['Area']=area

In [84]:
print("Total expenses ",expenses)
print("Leftover Budget ",b-expenses)
decider

Total expenses  76328973437.23828
Leftover Budget  23671026562.76172


,lat,long,Choice,Cost,Area
0,27.231708,70.303710,Wind,2.186692e+10,686
1,15.621395,73.876465,Wind,2.711702e+10,869
2,17.668997,79.586060,Solar,2.734503e+10,966
